<a href="https://colab.research.google.com/github/adrianwu8516/PersonalQuantResearch/blob/main/TQQQLab_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === 修復環境：安裝與 Colab 相容的版本，並強制重啟 ===
import sys, os, subprocess
def pipi(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

# 與 2025 Colab (Py3.11) 相容的安全組合
pipi(["numpy>=2.0,<2.2", "pandas>=2.2,<2.3", "scikit-learn>=1.5.1,<1.6",
      "gdown==5.2.0", "google-api-python-client>=2.120", "google-auth-oauthlib>=1.2"])

# 強制重啟，讓新版載入
os.kill(os.getpid(), 9)


In [5]:
# ============================================================
# 一鍵：Drive下載 → 訓練（OOF）→ 2D網格掃描(τ_CAGR) → S1/S2 CSV → 上傳
# 目標：以 CAGR 最大為主，若多組並列則挑 MaxDD 最小
# ============================================================

# ---- 參數 ----
SOURCE_DRIVE_FILE_ID = "1mDh7tLjcYV2_l2PZmY3_2JfgakGJLi2Y"
DESTINATION_FOLDER_ID = "1QhdQskcFWmODDxxtRjcXzUTK0Y8ckcfh"
TAU_MIN, TAU_MAX, TAU_STEP = 0.30, 0.80, 0.01
CAGR_TOL = 1e-4  # 並列判定：在 (maxCAGR - CAGR_TOL) 內視為同等，再取 MaxDD 最小

# ---- 套件不動 numpy/pandas/sklearn 主版本，補齊缺的 ----
import sys, subprocess, importlib, os, io, re, json, math, zipfile, tarfile, numpy as np, pandas as pd
def ensure(pkg, spec=None):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", spec or pkg])
ensure("google-api-python-client", "google-api-python-client>=2.120")
ensure("google-auth-oauthlib", "google-auth-oauthlib>=1.2")
ensure("scikit-learn", "scikit-learn>=1.5.1,<1.6")

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import average_precision_score, f1_score

SEED = 42
np.random.seed(SEED)
DATA_DIR, REPORT_DIR = "/content/data", "/content/reports"
os.makedirs(DATA_DIR, exist_ok=True); os.makedirs(REPORT_DIR, exist_ok=True)
drive = build("drive", "v3")

# ---------- Drive 下載（支援 檔案/資料夾/Shortcut/試算表） ----------
def drive_get_meta(file_id):
    return drive.files().get(fileId=file_id, fields="id,name,mimeType,shortcutDetails").execute()

def drive_resolve_id(file_id):
    m = drive_get_meta(file_id)
    if m.get("mimeType") == "application/vnd.google-apps.shortcut":
        tgt = m.get("shortcutDetails", {}).get("targetId")
        if not tgt: raise RuntimeError("Shortcut 無 targetId")
        return drive_resolve_id(tgt)
    return m

def drive_list_children(folder_id):
    items=[]; token=None
    while True:
        resp=drive.files().list(q=f"'{folder_id}' in parents and trashed=false",
                                fields="files(id,name,mimeType),nextPageToken",
                                pageToken=token).execute()
        items+=resp.get("files",[]); token=resp.get("nextPageToken")
        if not token: break
    return items

def download_file_binary(file_id, out_path):
    req = drive.files().get_media(fileId=file_id)
    with io.FileIO(out_path, "wb") as fh:
        dl = MediaIoBaseDownload(fh, req); done=False
        while not done:
            status, done = dl.next_chunk()
    return out_path

def download_sheet_csv(file_id, out_path):
    req = drive.files().export_media(fileId=file_id, mimeType="text/csv")
    with io.FileIO(out_path, "wb") as fh:
        dl = MediaIoBaseDownload(fh, req); done=False
        while not done:
            status, done = dl.next_chunk()
    return out_path

def drive_download_any(file_id, out_dir):
    m = drive_resolve_id(file_id)
    mt, name, fid = m["mimeType"], m["name"], m["id"]
    if mt == "application/vnd.google-apps.folder":
        paths=[]
        for it in drive_list_children(fid):
            paths += drive_download_any(it["id"], out_dir)
        return paths
    if mt == "application/vnd.google-apps.spreadsheet":
        p = os.path.join(out_dir, name if name.lower().endswith(".csv") else f"{name}.csv")
        return [download_sheet_csv(fid, p)]
    p = os.path.join(out_dir, name)
    return [download_file_binary(fid, p)]

def maybe_extract(path, out_dir):
    low = path.lower()
    if low.endswith(".zip"):
        with zipfile.ZipFile(path,"r") as z: z.extractall(out_dir); return True
    if low.endswith((".tar",".tar.gz",".tgz")):
        mode="r:gz" if low.endswith((".tar.gz",".tgz")) else "r"
        with tarfile.open(path,mode) as t: t.extractall(out_dir); return True
    return False

downloaded = drive_download_any(SOURCE_DRIVE_FILE_ID, DATA_DIR)
for p in downloaded:
    try: maybe_extract(p, DATA_DIR)
    except: pass
print("已下載：", downloaded)

# ---------- 載入五檔 or 單檔整合 ----------
REQ_FILES = {
    "TQQQ": "TQQQ_daily_OHLC_observations_latest.csv",
    "QQQ" : "QQQ_daily_OHLC_observations_latest.csv",
    "SPY" : "SPY_daily_OHLC_observations_latest.csv",
    "SPXL": "SPXL_daily_OHLC_observations_latest.csv",
    "VIXY": "VIXY_daily_OHLC_observations_latest.csv",
}
def _read_one(path, prefix):
    df = pd.read_csv(path)
    date_col = next((c for c in df.columns if c.lower() in ("date","datetime")), df.columns[0])
    df[date_col] = pd.to_datetime(df[date_col]); df=df.sort_values(date_col).set_index(date_col)
    cand=[c for c in df.columns if c.lower().replace(" ","") in ("adjclose","adj_close","adjustedclose")]
    if not cand: cand=[c for c in df.columns if c.lower()=="close"]
    if not cand: raise ValueError(f"{path} 找不到 Adj Close/Close")
    return df[[cand[0]]].rename(columns={cand[0]:f"{prefix}_AdjClose"})

def have_all_five(d):
    return all(os.path.exists(os.path.join(d,nm)) for nm in REQ_FILES.values())

def try_read_consolidated_csv(p):
    df = pd.read_csv(p)
    date_col = next((c for c in df.columns if c.lower() in ("date","datetime")), df.columns[0])
    df[date_col] = pd.to_datetime(df[date_col]); df=df.sort_values(date_col).set_index(date_col)
    cols={}
    for sym in ["TQQQ","QQQ","SPY","SPXL","VIXY"]:
        cand=[c for c in df.columns if re.fullmatch(fr"{sym}_?Adj(?:ust(ed)?)?_?Close", c, flags=re.I)]
        if not cand:
            cand=[c for c in df.columns if (sym in c) and (("adj" in c.lower()) or c.lower().endswith("close"))]
        if not cand: return None
        cols[sym]=cand[0]
    out=pd.DataFrame(index=df.index)
    for sym,c in cols.items(): out[f"{sym}_AdjClose"]=df[c].astype(float)
    return out

if have_all_five(DATA_DIR):
    parts=[_read_one(os.path.join(DATA_DIR,f), sym) for sym,f in REQ_FILES.items()]
    prices=pd.concat(parts,axis=1,join="inner").dropna()
else:
    ok=None
    for p in [os.path.join(DATA_DIR,f) for f in os.listdir(DATA_DIR) if f.lower().endswith(".csv")]:
        res=try_read_consolidated_csv(p)
        if res is not None: ok=res; break
    if ok is None: raise FileNotFoundError("無法解析來源 CSV")
    prices=ok

print("Data range:", prices.index.min().date(),"->",prices.index.max().date(),"shape:",prices.shape)

# ---------- 標籤 ----------
H, DD = 10, 0.15
K, BUY_DD, RB = 20, 0.20, 0.10
def make_labels(prices):
    p = prices["TQQQ_AdjClose"].copy()
    fut_min = p.shift(-1).rolling(H).min()
    y_warn = ((fut_min/p - 1.0) <= -DD).astype(int)
    rolling_max = p.cummax()
    cur_dd = p/rolling_max - 1.0
    fut_max = p.shift(-1).rolling(K).max()
    y_buy = ((cur_dd <= -BUY_DD) & ((fut_max/p - 1.0) >= RB)).astype(int)
    return y_warn, y_buy
y_warn, y_buy = make_labels(prices)

# ---------- 特徵（當日統一 shift(1)，再加 lag1/lag2；最後 dropna） ----------
def make_features(prices):
    feats=[]
    for sym in ["TQQQ","QQQ","SPY","SPXL","VIXY"]:
        s=prices[f"{sym}_AdjClose"]
        f=pd.DataFrame(index=prices.index)
        f[f"{sym}_r1"]=s.pct_change(1)
        f[f"{sym}_r5"]=s.pct_change(5)
        f[f"{sym}_ma10_ratio"]=s/s.rolling(10).mean()-1.0
        f[f"{sym}_ma20_ratio"]=s/s.rolling(20).mean()-1.0
        feats.append(f)
    base=pd.concat(feats,axis=1)
    base["TQQQ_QQQ_ratio"]=prices["TQQQ_AdjClose"]/prices["QQQ_AdjClose"]-1.0
    base["QQQ_SPY_ratio"]=prices["QQQ_AdjClose"]/prices["SPY_AdjClose"]-1.0
    lag1=base.add_suffix("_lag1").shift(1)
    lag2=base.add_suffix("_lag2").shift(2)
    return pd.concat([base.shift(1),lag1,lag2],axis=1).dropna()

X = make_features(prices)
df_all = pd.concat([X, y_warn.rename("y_warn"), y_buy.rename("y_buy")], axis=1).dropna()
X, y_warn, y_buy = df_all.drop(columns=["y_warn","y_buy"]), df_all["y_warn"].astype(int), df_all["y_buy"].astype(int)
dates = df_all.index
price = prices.loc[dates,"TQQQ_AdjClose"]

# ---------- OOF 機率（TimeSeriesSplit=5） ----------
def oof_prob(model, X, y, splits=5):
    tscv=TimeSeriesSplit(n_splits=splits)
    oof=pd.Series(index=X.index,dtype=float)
    for tr,te in tscv.split(X):
        mdl=model
        mdl.fit(X.iloc[tr], y.iloc[tr])
        oof.iloc[te]=mdl.predict_proba(X.iloc[te])[:,1]
    return oof

def pr_auc_masked(y, p):
    s=p.dropna()
    return average_precision_score(y.loc[s.index], s)

warn_models = {
    "PCA_Logistic": Pipeline([
        ("scaler", StandardScaler()),
        ("pca",    PCA(n_components=0.95, svd_solver="full", random_state=SEED)),
        ("logit",  LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=3000, random_state=SEED))
    ]),
    "Logistic_Elastic": Pipeline([
        ("scaler", StandardScaler()),
        ("logit",  LogisticRegression(penalty="elasticnet", l1_ratio=0.5, solver="saga",
                                      C=1.0, max_iter=5000, random_state=SEED))
    ]),
    "Logistic_L1": Pipeline([
        ("scaler", StandardScaler()),
        ("logit",  LogisticRegression(penalty="l1", solver="saga",
                                      C=1.0, max_iter=5000, random_state=SEED))
    ]),
}
buy_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logit",  LogisticRegression(penalty="l2", solver="lbfgs",
                                  C=1.0, max_iter=3000, random_state=SEED))
])

warn_probs = {n:oof_prob(m, X, y_warn, 5) for n,m in warn_models.items()}
p_warn = (warn_probs["PCA_Logistic"]+warn_probs["Logistic_Elastic"]+warn_probs["Logistic_L1"])/3.0
p_buy  = oof_prob(buy_model, X, y_buy, 5)

print("OOF NaN 數量：", {k:int(v.isna().sum()) for k,v in warn_probs.items()}, "| ensemble:", int(p_warn.isna().sum()), "| buy:", int(p_buy.isna().sum()))
print("WARN PR-AUCs:", {k: round(pr_auc_masked(y_warn, v),4) for k,v in warn_probs.items()})
print("WARN ensemble PR-AUC:", round(pr_auc_masked(y_warn, p_warn),4))
print("BUY  PR-AUC:", round(pr_auc_masked(y_buy,  p_buy ),4))

# ---------- 回測（S1 / S2） ----------
def backtest_s1(p_warn, tau_warn, price):
    ret_fwd = price.pct_change().shift(-1)
    pos = (p_warn < tau_warn).astype(int)   # NaN -> False -> 現金
    eq = (1 + pos*ret_fwd).cumprod()
    years=(price.index[-1]-price.index[0]).days/365.25
    cagr = eq.iloc[-2]**(1/years)-1
    maxdd = (eq/eq.cummax()-1).min()
    return float(cagr), float(maxdd)

def backtest_s2(p_warn, tau_warn, p_buy, tau_buy, price):
    ret_fwd = price.pct_change().shift(-1)
    base = (p_warn < tau_warn).astype(int)
    pos  = ((p_buy >= tau_buy) | (base==1)).astype(int)  # NaN 視為 False
    pos_shift = pos.shift(1).fillna(0)
    signal = np.where((pos==1)&(pos_shift==0),"BUY", np.where((pos==0)&(pos_shift==1),"SELL",""))
    reason = np.where(signal=="SELL","WARN",
              np.where((signal=="BUY")&(p_buy>=tau_buy),"BUY_DIP",
                       np.where(signal=="BUY","CLEAR_WARN","")))
    eq=(1 + pos*ret_fwd).cumprod()
    bench=(1 + ret_fwd).cumprod()
    years=(price.index[-1]-price.index[0]).days/365.25
    cagr=eq.iloc[-2]**(1/years)-1
    maxdd=(eq/eq.cummax()-1).min()
    report=pd.DataFrame({
        "Date": price.index, "TQQQ_AdjClose": price.values,
        "P_warn": p_warn.reindex(price.index), "P_buy": p_buy.reindex(price.index),
        "Position": pos.values, "Signal": signal, "Reason": reason,
        "Equity_S2": eq.values, "Equity_BH": bench.values
    }).dropna(subset=["TQQQ_AdjClose"]).reset_index(drop=True)
    return report, float(cagr), float(maxdd)

# ---------- τ 掃描（以 CAGR 最大；並列時取 MaxDD 最小） ----------
taus = np.round(np.arange(TAU_MIN, TAU_MAX + 1e-9, TAU_STEP), 2)

# S1：一維掃描
s1_grid=[]
for tw in taus:
    cagr,dd=backtest_s1(p_warn, tw, price)
    s1_grid.append((tw,cagr,dd))
s1_df = pd.DataFrame(s1_grid, columns=["tau_warn","CAGR","MaxDD"])
s1_max_cagr = s1_df["CAGR"].max()
s1_best = s1_df[(s1_df["CAGR"] >= s1_max_cagr - CAGR_TOL)].sort_values(["CAGR","MaxDD"], ascending=[False, True]).iloc[0]
tau_warn_best_s1 = float(s1_best["tau_warn"])

# S2：二維掃描
best_rec=None; best_cagr=-1; best_dd=0
for tw in taus:
    for tb in taus:
        _, cagr, dd = backtest_s2(p_warn, tw, p_buy, tb, price)
        if (cagr > best_cagr + 1e-12) or (abs(cagr - best_cagr) <= CAGR_TOL and dd > best_dd):
            best_cagr, best_dd, best_rec = cagr, dd, (tw, tb)
tau_warn_best_s2, tau_buy_best_s2 = map(float, best_rec)

# ---------- 產出報表（用最優 τ 設定） ----------
# S1
cagr_s1, dd_s1 = backtest_s1(p_warn, tau_warn_best_s1, price)
def export_s1(p_warn, tau_warn, price):
    ret_fwd = price.pct_change().shift(-1)
    pos = (p_warn < tau_warn).astype(int)
    pos_shift = pos.shift(1).fillna(0)
    signal = np.where((pos==1)&(pos_shift==0),"BUY", np.where((pos==0)&(pos_shift==1),"SELL",""))
    eq=(1+pos*ret_fwd).cumprod(); bench=(1+ret_fwd).cumprod()
    return pd.DataFrame({
        "Date": price.index, "TQQQ_AdjClose": price.values,
        "P_warn": p_warn.reindex(price.index),
        "Position": pos.values, "Signal": signal, "Reason": np.where(signal=="SELL","WARN",""),
        "Equity_S1": eq.values, "Equity_BH": bench.values
    }).dropna(subset=["TQQQ_AdjClose"]).reset_index(drop=True)
s1_report = export_s1(p_warn, tau_warn_best_s1, price)

# S2
s2_report, cagr_s2, dd_s2 = backtest_s2(p_warn, tau_warn_best_s2, p_buy, tau_buy_best_s2, price)

# 儲存
s1_path, s2_path = os.path.join(REPORT_DIR,"s1_trading_signals.csv"), os.path.join(REPORT_DIR,"s2_trading_signals.csv")
s1_report.to_csv(s1_path, index=False); s2_report.to_csv(s2_path, index=False)
print("本地輸出：", s1_path, s2_path)

# 上傳
def upload_csv(path, folder_id):
    meta={"name": os.path.basename(path), "parents":[folder_id]}
    media=MediaFileUpload(path, mimetype="text/csv", resumable=True)
    fid=drive.files().create(body=meta, media_body=media, fields="id").execute()["id"]
    print(f"已上傳：{os.path.basename(path)} -> https://drive.google.com/file/d/{fid}/view")
try:
    upload_csv(s1_path, DESTINATION_FOLDER_ID)
    upload_csv(s2_path, DESTINATION_FOLDER_ID)
except Exception as e:
    print("⚠️ 上傳失敗/跳過：", e)

summary = {
    "tau_warn_S1": tau_warn_best_s1,
    "S1_CAGR": float(cagr_s1), "S1_MaxDD": float(dd_s1),

    "tau_warn_S2": tau_warn_best_s2,
    "tau_buy_S2":  tau_buy_best_s2,
    "S2_CAGR": float(cagr_s2), "S2_MaxDD": float(dd_s2),

    "PR_AUC_warn_ensemble": float(pr_auc_masked(y_warn, p_warn)),
    "PR_AUC_buy_LogitL2": float(pr_auc_masked(y_buy, p_buy)),
    "date_start": str(price.index[0].date()), "date_end": str(price.index[-1].date())
}
print(summary)

已下載： ['/content/data/VIXY_daily_OHLC_observations_latest.csv', '/content/data/SPY_daily_OHLC_observations_latest.csv', '/content/data/QQQ_daily_OHLC_observations_latest.csv', '/content/data/TQQQ_daily_OHLC_observations_latest.csv', '/content/data/SPXL_daily_OHLC_observations_latest.csv']
Data range: 2011-01-04 -> 2025-08-15 shape: (3676, 5)
OOF NaN 數量： {'PCA_Logistic': 610, 'Logistic_Elastic': 610, 'Logistic_L1': 610} | ensemble: 610 | buy: 610
WARN PR-AUCs: {'PCA_Logistic': np.float64(0.5328), 'Logistic_Elastic': np.float64(0.538), 'Logistic_L1': np.float64(0.5344)}
WARN ensemble PR-AUC: 0.5484
BUY  PR-AUC: 0.8156
本地輸出： /content/reports/s1_trading_signals.csv /content/reports/s2_trading_signals.csv
已上傳：s1_trading_signals.csv -> https://drive.google.com/file/d/1RtgIT3A81K6BQDJ2LPnBn1E_Vk8_He92/view
已上傳：s2_trading_signals.csv -> https://drive.google.com/file/d/19zOWPHUi4mJaMU9TSe27w-CjMmkNcHVA/view
{'tau_warn_S1': 0.32, 'S1_CAGR': 0.4536479215493674, 'S1_MaxDD': -0.7383309908860027, 'ta

In [6]:
# ============================================================
# 一鍵：Drive下載 → OOF 訓練 → τ 掃描(以 CAGR 最大、同分取 MaxDD 最小) → 輸出多版本 S2
# 變體：
#   - S2_auto：全域掃描
#   - S2_strict：tau_buy 掃描縮窄到 0.77~0.85（避免過早進場）
#   - S2_cooldown：加入賣出之後的冷卻期 N 天，避免剛賣就買回
# ============================================================

# ---- 參數 ----
SOURCE_DRIVE_FILE_ID = "1mDh7tLjcYV2_l2PZmY3_2JfgakGJLi2Y"     # 來源：資料夾/檔案/Shortcut/試算表皆可
DESTINATION_FOLDER_ID = "1QhdQskcFWmODDxxtRjcXzUTK0Y8ckcfh"    # 目的地：Drive 資料夾

# 掃描範圍（一般）
TAU_WARN_RANGE = (0.30, 0.80, 0.01)
TAU_BUY_RANGE  = (0.30, 0.80, 0.01)

# 嚴格買點（Strict Buy，避免過早進場）
TAU_BUY_STRICT_RANGE = (0.77, 0.85, 0.01)   # 你可微調到 0.80±0.03

# 冷卻期（Cooldown）：賣出後 N 天內禁用 buy 強制進場；0 表示不啟用
COOLDOWN_DAYS = 3

CAGR_TOL = 1e-4  # 並列時的 CAGR 容差（同等視為並列，取 MaxDD 最小）

# ---- 套件（不動 numpy/pandas/sklearn 主版本，僅補齊缺的）----
import sys, subprocess, importlib, os, io, re, json, math, zipfile, tarfile, numpy as np, pandas as pd
def ensure(pkg, spec=None):
    try:
        importlib.import_module(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", spec or pkg])
ensure("google-api-python-client", "google-api-python-client>=2.120")
ensure("google-auth-oauthlib", "google-auth-oauthlib>=1.2")
ensure("scikit-learn", "scikit-learn>=1.5.1,<1.6")

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import average_precision_score

SEED = 42
np.random.seed(SEED)
DATA_DIR, REPORT_DIR = "/content/data", "/content/reports"
os.makedirs(DATA_DIR, exist_ok=True); os.makedirs(REPORT_DIR, exist_ok=True)
drive = build("drive", "v3")

# ---------- Drive 下載（支援 檔案/資料夾/Shortcut/試算表） ----------
def drive_get_meta(file_id):
    return drive.files().get(fileId=file_id, fields="id,name,mimeType,shortcutDetails").execute()

def drive_resolve_id(file_id):
    m = drive_get_meta(file_id)
    if m.get("mimeType") == "application/vnd.google-apps.shortcut":
        tgt = m.get("shortcutDetails", {}).get("targetId")
        if not tgt: raise RuntimeError("Shortcut 無 targetId")
        return drive_resolve_id(tgt)
    return m

def drive_list_children(folder_id):
    items=[]; token=None
    while True:
        resp=drive.files().list(q=f"'{folder_id}' in parents and trashed=false",
                                fields="files(id,name,mimeType),nextPageToken",
                                pageToken=token).execute()
        items+=resp.get("files",[]); token=resp.get("nextPageToken")
        if not token: break
    return items

def download_file_binary(file_id, out_path):
    req = drive.files().get_media(fileId=file_id)
    with io.FileIO(out_path, "wb") as fh:
        dl = MediaIoBaseDownload(fh, req); done=False
        while not done:
            status, done = dl.next_chunk()
    return out_path

def download_sheet_csv(file_id, out_path):
    req = drive.files().export_media(fileId=file_id, mimeType="text/csv")
    with io.FileIO(out_path, "wb") as fh:
        dl = MediaIoBaseDownload(fh, req); done=False
        while not done:
            status, done = dl.next_chunk()
    return out_path

def drive_download_any(file_id, out_dir):
    m = drive_resolve_id(file_id)
    mt, name, fid = m["mimeType"], m["name"], m["id"]
    if mt == "application/vnd.google-apps.folder":
        paths=[]
        for it in drive_list_children(fid):
            paths += drive_download_any(it["id"], out_dir)
        return paths
    if mt == "application/vnd.google-apps.spreadsheet":
        p = os.path.join(out_dir, name if name.lower().endswith(".csv") else f"{name}.csv")
        return [download_sheet_csv(fid, p)]
    p = os.path.join(out_dir, name)
    return [download_file_binary(fid, p)]

def maybe_extract(path, out_dir):
    low = path.lower()
    if low.endswith(".zip"):
        with zipfile.ZipFile(path,"r") as z: z.extractall(out_dir); return True
    if low.endswith((".tar",".tar.gz",".tgz")):
        mode="r:gz" if low.endswith((".tar.gz",".tgz")) else "r"
        with tarfile.open(path,mode) as t: t.extractall(out_dir); return True
    return False

downloaded = drive_download_any(SOURCE_DRIVE_FILE_ID, DATA_DIR)
for p in downloaded:
    try: maybe_extract(p, DATA_DIR)
    except: pass
print("已下載：", downloaded)

# ---------- 載入五檔 or 單檔整合 ----------
REQ_FILES = {
    "TQQQ": "TQQQ_daily_OHLC_observations_latest.csv",
    "QQQ" : "QQQ_daily_OHLC_observations_latest.csv",
    "SPY" : "SPY_daily_OHLC_observations_latest.csv",
    "SPXL": "SPXL_daily_OHLC_observations_latest.csv",
    "VIXY": "VIXY_daily_OHLC_observations_latest.csv",
}
def _read_one(path, prefix):
    df = pd.read_csv(path)
    date_col = next((c for c in df.columns if c.lower() in ("date","datetime")), df.columns[0])
    df[date_col] = pd.to_datetime(df[date_col]); df=df.sort_values(date_col).set_index(date_col)
    cand=[c for c in df.columns if c.lower().replace(" ","") in ("adjclose","adj_close","adjustedclose")]
    if not cand: cand=[c for c in df.columns if c.lower()=="close"]
    if not cand: raise ValueError(f"{path} 找不到 Adj Close/Close")
    return df[[cand[0]]].rename(columns={cand[0]:f"{prefix}_AdjClose"})

def have_all_five(d):
    return all(os.path.exists(os.path.join(d,nm)) for nm in REQ_FILES.values())

def try_read_consolidated_csv(p):
    df = pd.read_csv(p)
    date_col = next((c for c in df.columns if c.lower() in ("date","datetime")), df.columns[0])
    df[date_col] = pd.to_datetime(df[date_col]); df=df.sort_values(date_col).set_index(date_col)
    cols={}
    for sym in ["TQQQ","QQQ","SPY","SPXL","VIXY"]:
        cand=[c for c in df.columns if re.fullmatch(fr"{sym}_?Adj(?:ust(ed)?)?_?Close", c, flags=re.I)]
        if not cand:
            cand=[c for c in df.columns if (sym in c) and (("adj" in c.lower()) or c.lower().endswith("close"))]
        if not cand: return None
        cols[sym]=cand[0]
    out=pd.DataFrame(index=df.index)
    for sym,c in cols.items(): out[f"{sym}_AdjClose"]=df[c].astype(float)
    return out

if have_all_five(DATA_DIR):
    parts=[_read_one(os.path.join(DATA_DIR,f), sym) for sym,f in REQ_FILES.items()]
    prices=pd.concat(parts,axis=1,join="inner").dropna()
else:
    ok=None
    for p in [os.path.join(DATA_DIR,f) for f in os.listdir(DATA_DIR) if f.lower().endswith(".csv")]:
        res=try_read_consolidated_csv(p)
        if res is not None: ok=res; break
    if ok is None: raise FileNotFoundError("無法解析來源 CSV")
    prices=ok

print("Data range:", prices.index.min().date(),"->",prices.index.max().date(),"shape:",prices.shape)

# ---------- 標籤 ----------
H, DD = 10, 0.15
K, BUY_DD, RB = 20, 0.20, 0.10
def make_labels(prices):
    p = prices["TQQQ_AdjClose"].copy()
    fut_min = p.shift(-1).rolling(H).min()
    y_warn = ((fut_min/p - 1.0) <= -DD).astype(int)
    rolling_max = p.cummax()
    cur_dd = p/rolling_max - 1.0
    fut_max = p.shift(-1).rolling(K).max()
    y_buy = ((cur_dd <= -BUY_DD) & ((fut_max/p - 1.0) >= RB)).astype(int)
    return y_warn, y_buy
y_warn, y_buy = make_labels(prices)

# ---------- 特徵（當日統一 shift(1)，再加 lag1/lag2；最後 dropna） ----------
def make_features(prices):
    feats=[]
    for sym in ["TQQQ","QQQ","SPY","SPXL","VIXY"]:
        s=prices[f"{sym}_AdjClose"]
        f=pd.DataFrame(index=prices.index)
        f[f"{sym}_r1"]=s.pct_change(1)
        f[f"{sym}_r5"]=s.pct_change(5)
        f[f"{sym}_ma10_ratio"]=s/s.rolling(10).mean()-1.0
        f[f"{sym}_ma20_ratio"]=s/s.rolling(20).mean()-1.0
        feats.append(f)
    base=pd.concat(feats,axis=1)
    base["TQQQ_QQQ_ratio"]=prices["TQQQ_AdjClose"]/prices["QQQ_AdjClose"]-1.0
    base["QQQ_SPY_ratio"]=prices["QQQ_AdjClose"]/prices["SPY_AdjClose"]-1.0
    lag1=base.add_suffix("_lag1").shift(1)
    lag2=base.add_suffix("_lag2").shift(2)
    return pd.concat([base.shift(1),lag1,lag2],axis=1).dropna()

X = make_features(prices)
df_all = pd.concat([X, y_warn.rename("y_warn"), y_buy.rename("y_buy")], axis=1).dropna()
X, y_warn, y_buy = df_all.drop(columns=["y_warn","y_buy"]), df_all["y_warn"].astype(int), df_all["y_buy"].astype(int)
dates = df_all.index
price = prices.loc[dates,"TQQQ_AdjClose"]

# ---------- OOF 機率（TimeSeriesSplit=5；OOF NaN 頭段→現金） ----------
def oof_prob(model, X, y, splits=5):
    tscv=TimeSeriesSplit(n_splits=splits)
    oof=pd.Series(index=X.index,dtype=float)
    for tr,te in tscv.split(X):
        mdl=model
        mdl.fit(X.iloc[tr], y.iloc[tr])
        oof.iloc[te]=mdl.predict_proba(X.iloc[te])[:,1]
    return oof

def pr_auc_masked(y, p):
    s=p.dropna()
    return average_precision_score(y.loc[s.index], s)

warn_models = {
    "PCA_Logistic": Pipeline([
        ("scaler", StandardScaler()),
        ("pca",    PCA(n_components=0.95, svd_solver="full", random_state=SEED)),
        ("logit",  LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=3000, random_state=SEED))
    ]),
    "Logistic_Elastic": Pipeline([
        ("scaler", StandardScaler()),
        ("logit",  LogisticRegression(penalty="elasticnet", l1_ratio=0.5, solver="saga",
                                      C=1.0, max_iter=5000, random_state=SEED))
    ]),
    "Logistic_L1": Pipeline([
        ("scaler", StandardScaler()),
        ("logit",  LogisticRegression(penalty="l1", solver="saga",
                                      C=1.0, max_iter=5000, random_state=SEED))
    ]),
}
buy_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logit",  LogisticRegression(penalty="l2", solver="lbfgs",
                                  C=1.0, max_iter=3000, random_state=SEED))
])

warn_probs = {n:oof_prob(m, X, y_warn, 5) for n,m in warn_models.items()}
p_warn = (warn_probs["PCA_Logistic"]+warn_probs["Logistic_Elastic"]+warn_probs["Logistic_L1"])/3.0
p_buy  = oof_prob(buy_model, X, y_buy, 5)

print("OOF NaN 數量：", {k:int(v.isna().sum()) for k,v in warn_probs.items()}, "| ensemble:", int(p_warn.isna().sum()), "| buy:", int(p_buy.isna().sum()))
print("WARN PR-AUCs:", {k: round(pr_auc_masked(y_warn, v),4) for k,v in warn_probs.items()})
print("WARN ensemble PR-AUC:", round(pr_auc_masked(y_warn, p_warn),4))
print("BUY  PR-AUC:", round(pr_auc_masked(y_buy,  p_buy ),4))

# ---------- 回測（S1 / S2，含冷卻期版本） ----------
def backtest_s1(p_warn, tau_warn, price):
    ret_fwd = price.pct_change().shift(-1)
    pos = (p_warn < tau_warn).astype(int)   # NaN -> False -> 現金
    eq = (1 + pos*ret_fwd).cumprod()
    years=(price.index[-1]-price.index[0]).days/365.25
    cagr = eq.iloc[-2]**(1/years)-1
    maxdd = (eq/eq.cummax()-1).min()
    return float(cagr), float(maxdd)

def backtest_s2(p_warn, tau_warn, p_buy, tau_buy, price, cooldown_days=0):
    ret_fwd = price.pct_change().shift(-1)
    base = (p_warn < tau_warn).astype(int)              # warn 未觸發 → 滿倉
    force_buy = (p_buy >= tau_buy).astype(int)          # buy 強制持有
    if cooldown_days and cooldown_days>0:
        sold = (base.shift(1).fillna(0)==1) & (base==0)  # 從持有→現金 的轉折日
        cool = sold.copy()
        for k in range(1, cooldown_days):
            cool = cool | sold.shift(k).fillna(False)
        force_buy = force_buy & (~cool)                 # 冷卻期內禁止 buy 強制進場
    pos  = ((base==1) | (force_buy==1)).astype(int)
    pos_shift = pos.shift(1).fillna(0)
    signal = np.where((pos==1)&(pos_shift==0),"BUY", np.where((pos==0)&(pos_shift==1),"SELL",""))
    reason = np.where(signal=="SELL","WARN",
              np.where((signal=="BUY")&(p_buy>=tau_buy),"BUY_DIP",
                       np.where(signal=="BUY","CLEAR_WARN","")))
    eq=(1 + pos*ret_fwd).cumprod()
    bench=(1 + ret_fwd).cumprod()
    years=(price.index[-1]-price.index[0]).days/365.25
    cagr=eq.iloc[-2]**(1/years)-1
    maxdd=(eq/eq.cummax()-1).min()
    report=pd.DataFrame({
        "Date": price.index, "TQQQ_AdjClose": price.values,
        "P_warn": p_warn.reindex(price.index), "P_buy": p_buy.reindex(price.index),
        "Position": pos.values, "Signal": signal, "Reason": reason,
        "Equity_S2": eq.values, "Equity_BH": bench.values
    }).dropna(subset=["TQQQ_AdjClose"]).reset_index(drop=True)
    return report, float(cagr), float(maxdd)

# ---------- 掃描器（以 CAGR 最大，並列時取 MaxDD 最小） ----------
def scan_s1(p_warn, price, tau_range):
    lo, hi, step = tau_range
    taus = np.round(np.arange(lo, hi+1e-9, step), 2)
    grid=[]
    for tw in taus:
        cagr,dd = backtest_s1(p_warn, tw, price)
        grid.append((tw,cagr,dd))
    df = pd.DataFrame(grid, columns=["tau_warn","CAGR","MaxDD"])
    max_cagr = df["CAGR"].max()
    best = df[df["CAGR"] >= max_cagr - CAGR_TOL].sort_values(["CAGR","MaxDD"], ascending=[False, True]).iloc[0]
    return float(best["tau_warn"]), float(best["CAGR"]), float(best["MaxDD"]), df

def scan_s2(p_warn, p_buy, price, tau_warn_range, tau_buy_range, cooldown_days=0):
    lo_w, hi_w, step_w = tau_warn_range
    lo_b, hi_b, step_b = tau_buy_range
    taus_w = np.round(np.arange(lo_w, hi_w+1e-9, step_w), 2)
    taus_b = np.round(np.arange(lo_b, hi_b+1e-9, step_b), 2)
    best=None; best_cagr=-1; best_dd=-1
    recs=[]
    for tw in taus_w:
        for tb in taus_b:
            _, cagr, dd = backtest_s2(p_warn, tw, p_buy, tb, price, cooldown_days=cooldown_days)
            recs.append((tw, tb, cagr, dd))
            if (cagr > best_cagr + 1e-12) or (abs(cagr - best_cagr) <= CAGR_TOL and dd > best_dd):
                best_cagr, best_dd, best = cagr, dd, (tw,tb)
    df = pd.DataFrame(recs, columns=["tau_warn","tau_buy","CAGR","MaxDD"])
    return (float(best[0]), float(best[1]), float(best_cagr), float(best_dd), df)

# ---------- 執行掃描：S1 / S2 多版本 ----------
tau_w_s1, cagr_s1, dd_s1, s1_df = scan_s1(p_warn, price, TAU_WARN_RANGE)

# S2_auto：全域掃描
tw_auto, tb_auto, cagr_s2_auto, dd_s2_auto, _ = scan_s2(p_warn, p_buy, price, TAU_WARN_RANGE, TAU_BUY_RANGE, cooldown_days=0)

# S2_strict：僅 buy 嚴格範圍
tw_strict, tb_strict, cagr_s2_strict, dd_s2_strict, _ = scan_s2(p_warn, p_buy, price, TAU_WARN_RANGE, TAU_BUY_STRICT_RANGE, cooldown_days=0)

# S2_cooldown：全域掃描 + 冷卻期
tw_cd, tb_cd, cagr_s2_cd, dd_s2_cd, _ = scan_s2(p_warn, p_buy, price, TAU_WARN_RANGE, TAU_BUY_RANGE, cooldown_days=COOLDOWN_DAYS)

# ---------- 產出 CSV ----------
def export_s1(p_warn, tau_warn, price):
    ret_fwd = price.pct_change().shift(-1)
    pos = (p_warn < tau_warn).astype(int)
    pos_shift = pos.shift(1).fillna(0)
    signal = np.where((pos==1)&(pos_shift==0),"BUY", np.where((pos==0)&(pos_shift==1),"SELL",""))
    eq=(1+pos*ret_fwd).cumprod(); bench=(1+ret_fwd).cumprod()
    return pd.DataFrame({
        "Date": price.index, "TQQQ_AdjClose": price.values,
        "P_warn": p_warn.reindex(price.index),
        "Position": pos.values, "Signal": signal, "Reason": np.where(signal=="SELL","WARN",""),
        "Equity_S1": eq.values, "Equity_BH": bench.values
    }).dropna(subset=["TQQQ_AdjClose"]).reset_index(drop=True)

def export_s2(p_warn, tau_warn, p_buy, tau_buy, price, cooldown_days=0):
    report,_,_ = backtest_s2(p_warn, tau_warn, p_buy, tau_buy, price, cooldown_days=cooldown_days)
    return report

s1_path        = os.path.join(REPORT_DIR,"s1_trading_signals.csv")
s2_auto_path   = os.path.join(REPORT_DIR,"s2_auto_trading_signals.csv")
s2_strict_path = os.path.join(REPORT_DIR,"s2_strict_trading_signals.csv")
s2_cd_path     = os.path.join(REPORT_DIR,"s2_cooldown_trading_signals.csv")

export_s1(p_warn, tau_w_s1, price).to_csv(s1_path, index=False)
export_s2(p_warn, tw_auto,   p_buy, tb_auto,   price, cooldown_days=0).to_csv(s2_auto_path, index=False)
export_s2(p_warn, tw_strict, p_buy, tb_strict, price, cooldown_days=0).to_csv(s2_strict_path, index=False)
export_s2(p_warn, tw_cd,     p_buy, tb_cd,     price, cooldown_days=COOLDOWN_DAYS).to_csv(s2_cd_path, index=False)

print("本地輸出：", s1_path, s2_auto_path, s2_strict_path, s2_cd_path)

# ---------- 上傳到目的地資料夾 ----------
def upload_csv(path, folder_id):
    meta={"name": os.path.basename(path), "parents":[folder_id]}
    media=MediaFileUpload(path, mimetype="text/csv", resumable=True)
    fid=drive.files().create(body=meta, media_body=media, fields="id").execute()["id"]
    print(f"已上傳：{os.path.basename(path)} -> https://drive.google.com/file/d/{fid}/view")

try:
    for p in [s1_path, s2_auto_path, s2_strict_path, s2_cd_path]:
        upload_csv(p, DESTINATION_FOLDER_ID)
except Exception as e:
    print("⚠️ 上傳失敗/跳過：", e)

# ---------- 摘要輸出 ----------
summary = {
    "S1": {
        "tau_warn": tau_w_s1, "CAGR": float(cagr_s1), "MaxDD": float(dd_s1)
    },
    "S2_auto": {
        "tau_warn": tw_auto, "tau_buy": tb_auto,
        "CAGR": float(cagr_s2_auto), "MaxDD": float(dd_s2_auto)
    },
    "S2_strict": {
        "tau_warn": tw_strict, "tau_buy": tb_strict,
        "CAGR": float(cagr_s2_strict), "MaxDD": float(dd_s2_strict),
        "note": "buy 門檻範圍縮窄至 0.77~0.85，降低過早進場風險"
    },
    "S2_cooldown": {
        "tau_warn": tw_cd, "tau_buy": tb_cd, "cooldown_days": COOLDOWN_DAYS,
        "CAGR": float(cagr_s2_cd), "MaxDD": float(dd_s2_cd),
        "note": f"賣出後 {COOLDOWN_DAYS} 天內禁 buy 強制進場，降低來回進出"
    },
    "PR_AUC_warn_ensemble": float(pr_auc_masked(y_warn, p_warn)),
    "PR_AUC_buy_L2": float(pr_auc_masked(y_buy, p_buy)),
    "date_start": str(price.index[0].date()), "date_end": str(price.index[-1].date())
}
print(json.dumps(summary, indent=2))


已下載： ['/content/data/VIXY_daily_OHLC_observations_latest.csv', '/content/data/SPY_daily_OHLC_observations_latest.csv', '/content/data/QQQ_daily_OHLC_observations_latest.csv', '/content/data/TQQQ_daily_OHLC_observations_latest.csv', '/content/data/SPXL_daily_OHLC_observations_latest.csv']
Data range: 2011-01-04 -> 2025-08-15 shape: (3676, 5)
OOF NaN 數量： {'PCA_Logistic': 610, 'Logistic_Elastic': 610, 'Logistic_L1': 610} | ensemble: 610 | buy: 610
WARN PR-AUCs: {'PCA_Logistic': np.float64(0.5328), 'Logistic_Elastic': np.float64(0.538), 'Logistic_L1': np.float64(0.5344)}
WARN ensemble PR-AUC: 0.5484
BUY  PR-AUC: 0.8156


Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-202831141.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cool = cool | sold.shift(k).fillna(False)
/tmp/ipython-input-202831141.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cool = cool | sold.shift(k).fillna(False)
/tmp/ipython-input-202831141.py:271: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_opti

本地輸出： /content/reports/s1_trading_signals.csv /content/reports/s2_auto_trading_signals.csv /content/reports/s2_strict_trading_signals.csv /content/reports/s2_cooldown_trading_signals.csv
已上傳：s1_trading_signals.csv -> https://drive.google.com/file/d/1uK0W3hbtS48TiZOEBJRm4AabIycNFit9/view
已上傳：s2_auto_trading_signals.csv -> https://drive.google.com/file/d/1hyiTy2ME1fCRjqa6kNf424HLBMfd7U6q/view
已上傳：s2_strict_trading_signals.csv -> https://drive.google.com/file/d/1LGC1OEqiyb2hsWrlmbMJzmo2TQoqzaMp/view
已上傳：s2_cooldown_trading_signals.csv -> https://drive.google.com/file/d/1vtvep9JrvGMOBSrhTZcSnK3nKVg8fceL/view
{
  "S1": {
    "tau_warn": 0.32,
    "CAGR": 0.4536479215493674,
    "MaxDD": -0.7383309908860027
  },
  "S2_auto": {
    "tau_warn": 0.36,
    "tau_buy": 0.67,
    "CAGR": 0.42162498421912775,
    "MaxDD": -0.7535056271138473
  },
  "S2_strict": {
    "tau_warn": 0.33,
    "tau_buy": 0.85,
    "CAGR": 0.4271111158415746,
    "MaxDD": -0.7402174366654358,
    "note": "buy \u9580\u6abb